<a href="https://colab.research.google.com/github/liandrorefulle/PPL---125L-Project/blob/main/125L_PPL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1. Introduction to the Problem/Task and Interpreter System

An **interpreter system** is a program that directly executes instructions written in a programming or scripting language, without requiring them to be compiled into machine code first. This process involves reading source code, analyzing it (lexing and parsing), and then executing the commands to produce a result. Interpreters are vital in environments requiring high portability, rapid prototyping, and dynamic execution, such as command shells, scripting languages (like Python or JavaScript), and specialized domain-specific languages (DSLs).

The specific interpreter system chosen is a **Custom Scripting Language Interpreter**.

This project implements an interpreter for a domain-specific language (DSL) that supports **procedural control flow, variable management with explicit typing, and arithmetic evaluation**. The project was selected to provide a practical, hands-on understanding of the fundamental phases of interpreter design: **Lexical Analysis (Lexer), Syntactic Analysis (Parser), and Execution (Executor)**. The target task is the sequential evaluation and execution of statements written in the custom language, including mathematical calculations, conditional logic, and iterative control structures.

# Section 2. Description of the Input Language

The input language, hereafter referred to as **PPL-Script**, is a simple, block-structured, imperative language designed for performing basic arithmetic, managing explicitly-typed variables, and implementing fundamental control flow. Its design rationale is to mimic the structure of high-level scripting languages while enforcing explicit data types to reduce runtime ambiguity.

#### **Inspiration and Purpose**
PPL-Script is inspired by the syntactic clarity of languages like Python and the explicit type declaration found in static languages. It is designed to solve introductory programming problems focused on **arithmetic computation and flow control**, serving as a teaching tool to understand core programming constructs and interpreter operations.

#### **Language Structure**

| Category | Tokens / Keywords / Operators | Description |
| :--- | :--- | :--- |
| **Data Types** | `int`, `deci` (float), `bool`, **String** (in `"quotes"`) | Explicit type annotation for variables is supported (e.g., `num(int)`). |
| **I/O** | `read`, `print` | Input and output operations. Variable references use the `$` prefix in `print` statements. |
| **Assignment** | `=` | Used for variable definition and update. |
| **Arithmetic** | `+`, `-`, `*`, `/`, `%`, `^` (Exponent) | Standard arithmetic operations. Expressions are wrapped in square brackets `[]` for clarity. |
| **Comparison** | `==`, `!=`, `>`, `<`, `>=`, `<=` | Used in conditional expressions. |
| **Logical** | `&&` (AND), `||` (OR) | Boolean logic for compound conditions. |
| **Control Flow**| `if`, `elif`, `else`, `while`, `repeat`, `done` | Used for conditional execution and looping. Conditions/counts end with a colon `:`. Blocks are terminated by `done`. |
| **Metadata** | `(type)`, `[expression]`, `:` | Parentheses for explicit type, brackets for mathematical/logical expressions, and colon for block headers. |
| **Comments** | `//` | Single-line comments. |

#### **Examples**

| Type | Valid Input | Interpreter Response | Justification |
| :--- | :--- | :--- | :--- |
| **Valid** | `math(int) = [ 2 + 2 ]` | Variable `math` is assigned the integer value `4`. | Follows the `var(type) = [expr]` assignment syntax. |
| **Valid** | `if [x > 0]: print "Pos" done` | If expression evaluates to `true`, the `print` command executes. | Uses correct `if-done` block structure and bracketed condition. |
| **Invalid**| `num = 10` | Error: Invalid assignment syntax. | The language requires an explicit type annotation for non-string assignments (`num(int) = 10`). |
| **Invalid**| `while x < 5: print "Loop" done` | Error: Invalid condition syntax. | The condition `x < 5` is not enclosed in the mandatory square brackets `[]`. |

# Section 3. System Design

The interpreter is built entirely using standard Python libraries, prioritizing efficiency and minimizing external dependencies.

| Category | Library/Module | Purpose in Project |
| :--- | :--- | :--- |
| **Built-in** | `sys` | Used for handling command-line arguments (file path check) and program exit. |
| **Built-in** | `re` (Regular Expressions) | Crucial for **Lexical Analysis** (`tokenize` function) to identify comments, quoted strings, and distinct token types; also used in **Parsing** to extract variable names and types. |
| **Third-Party** | *(None)* | The implementation uses no third-party libraries. |

# Section 4. Data Preprocessing and Cleaning

#### **Overview of Interpreter Architecture**

The PPL-Script interpreter follows the classic three-stage architecture:

1.  **Lexer (Tokenizer):** Converts the input text (source code) into a linear sequence of recognized tokens.
2.  **Parser:** Takes the token stream and organizes it into a structured command representation (a dictionary/Abstract Syntax Tree proxy) that reflects the language's grammar.
3.  **Executor (Interpreter Engine):** Executes the structured commands by manipulating program state (the `variables` dictionary) and performing I/O.

#### **Data Flow Diagram**

The data flow is sequential and iterative for each line of code:

**Input (Source Code Line) → Tokenization (Lexer) → Parsing (Command Structure) → Execution (Executor) → Output/State Change**



1.  **Input:** A single line of PPL-Script is read from the source file.
2.  **Tokenization:** The `tokenize` function removes comments and uses regular expressions to capture distinct meaningful units (e.g., keywords, operators, values, variable names).
3.  **Parsing:** The `parse` function identifies the command type based on keywords (e.g., `print`, `read`) or structural elements (e.g., `=`). It generates a dictionary (`command`) containing the command and its arguments (`cmd`, `name`, `value`, `condition`, etc.).
4.  **Execution:** The `execute` function or the main `run` loop processes the `command` dictionary. Simple commands are executed directly, while control flow commands (`if`, `while`, `repeat`) manage the program counter (`i`) and delegate block execution to `run_block`.

#### **Error Handling Strategies**

The interpreter employs two primary error handling mechanisms:

1.  **File/Setup Errors:** The `run` function includes a `try-except` block to catch `FileNotFoundError` for the program file and handles command-line argument validation (`sys.argv` check).
2.  **Runtime/Syntax Errors:** A broad `try-except Exception as e` is used within the main loop of `run` to catch any error during tokenization, parsing, or execution, providing a message including the line number (`Error at line {i+1}: {e}`). Explicit **syntax validation** is performed within the `parse` and `evaluate` functions using `raise ValueError` for known invalid patterns (e.g., in `read` statement syntax or during expression evaluation).

#### **Design Justification (Critical Consideration)**

The interpreter uses a **keyword-based, top-down parsing method** implemented via sequential `if/elif` statements in the `parse` function. This approach is justified because:

* **Simplicity and Speed:** For a simple DSL with a limited grammar, this non-formal parsing method is fast to implement and easy to maintain.
* **Delegated Complexity:** The most complex part—mathematical and logical expression evaluation—is **delegated** to Python's built-in `eval()` function after pre-processing (variable substitution and operator translation). This saves significant development time by avoiding the need to write a full expression parser.

# Section 5. Implementation Details

Provide and explain the implementation of your interpreter step by step. Show the source code for each component:

#### **Lexer: The `tokenize` Function**

The `tokenize` function performs two key preprocessing steps: comment removal and token isolation.

```python
def tokenize(line):
    # remove comments
    line = re.sub(r"//.*", "", line).strip()
    if not line:
        return []
    # properly handle quoted strings and symbols
    return re.findall(r'\"[^\"]*\"|\w+\([^)]*?\)|==|!=|>=|<=|&&|\|\||[=+\\-*/%^<>:]|\\$?\\w+', line)
```

* **Comment Removal**: The function uses re.sub(r"//.*", "", line) to strip out all text that follows a single-line comment marker (//).

* **Token Recognition**: A comprehensive regular expression pattern uses the pipe (|) to prioritize capturing multi-character sequences (like comparison/logical operators) and structured elements (like quoted strings and typed variables: \w+\([^)]*?\)), ensuring tokens are delimited correctly.

####**Parser: The parse Function**

The parse function takes the token stream and converts it into a structured command dictionary, which acts as a simplified intermediate representation (IR). It uses the first token to determine the command type and then extracts relevant parameters.

```python
def parse(tokens):
    if not tokens:
        return None

    # print statement
    if tokens[0] == "print":
        return {"cmd": "print", "expr": " ".join(tokens[1:])}

    # variable declaration or math
    if "=" in tokens:
        left = tokens[0]
        value_index = tokens.index("=") + 1
        value = " ".join(tokens[value_index:])

    # support typed variable: var(type) = expr
        match = re.match(r"(\w+)\((\w+)\)", left)
        if match:
            varname, vartype = match.groups()
        else:
            varname, vartype = left, None

        return {"cmd": "assign", "name": varname, "type": vartype, "value": value}


    # read statement: read "Prompt" var(type)
    if tokens[0] == "read":
        # ... logic to extract prompt, var name, and type
        prompt = tokens[1].strip('\"')
        var_decl = tokens[2]
        match = re.match(r"(\w+)\((\w+)\)", var_decl)
        if not match:
            raise ValueError("Invalid read syntax. Use: read \"Prompt\" var(type)")
        var_name, var_type = match.groups()
        return {"cmd": "read", "prompt": prompt, "name": var_name, "type": var_type}

    # if / elif / else / done / while / repeat
    if tokens[0] in ["if", "elif", "else", "while", "repeat", "repeat_as"]:
        # ... logic for control flow commands extracts conditions/counts
        pass

    return {"cmd": "unknown", "tokens": tokens}

####**Executor: The evaluate and execute Functions**

The execution phase handles program state changes and output. This is accomplished via two main functions: evaluate for expressions, and execute for commands.

evaluate(expr): Expression Evaluation

This function prepares and executes mathematical or logical expressions by translating PPL-Script syntax into valid Python syntax and substituting variables before calling eval().

```python
def evaluate(expr):
    expr = expr.replace("^", "**")  # exponent operator
    expr = expr.replace("[", "").replace("]", "")
    for name, val in variables.items():
        # Replace $var and plain var names
        expr = expr.replace(f"${name}", str(val))
        expr = re.sub(rf"\\b{name}\\b", str(val), expr)

    try:
        return eval(expr)
    except Exception as e:
        raise ValueError(f"Invalid expression: {expr} ({e})")
```
* **Operator Mapping:** Translates the PPL-Script exponent operator ^ to Python's for compatibility with eval().

* **Variable Substitution:** Replaces all instances of $var and plain var within the expression string with their stored values retrieved from the global variables dictionary.


execute(command): Command Execution

This function handles the execution of non-control flow commands (assignment, print, read).
```python
def execute(command):
    cmd_type = command["cmd"]

    if cmd_type == "assign":
        val = evaluate(command["value"])
        if "type" in command and command["type"]:
            t = command["type"]
            # Perform explicit type casting based on declaration
            if t == "int":
                val = int(val)
            elif t in ["deci", "float"]:
                val = float(val)
            # ... other types
        variables[command["name"]] = val

    elif cmd_type == "print":
        text = command["expr"]
        for name, val in variables.items():
            text = text.replace(f"${name}", str(val))
        print(text.strip('\"'))

    elif cmd_type == "read":
        # ... Handles input and explicit type casting
        user_input = input(command["prompt"])
        # ... logic for type casting
        variables[command["name"]] = val
```

* **Assignment (assign):** After evaluating the right-hand expression, it performs the required explicit type casting (e.g., int(), float()) before storing the variable in the global variables dictionary.

* **Execution of Control Flow:** The control flow commands (if, while, repeat) are managed outside of execute in the main run loop, which handles reading lines, managing the program counter, and delegating block execution.

# Section 6. Testing with Valid and Invalid Inputs

Demonstrate how your interpreter works by running a variety of test cases. These tests prove the correctness of the Lexer and Parser, and the robustness of the Executor in handling different data types and control flows.

### **Valid Commands and Outputs**

These examples verify the successful processing of variable assignment, arithmetic, flow control, and string interpolation, confirming the interpreter's expected behavior.

#### **Test Case 6.1: Simple Assignment & Print**
* **PPL-Script Code:**
    ```
    length(deci) = 2
    width(deci) = 4
    area(deci) = [length * width]
    print "Area = $area"
    ```
* **Expected Output:** `Area = 8.0`
* **Correctness:** Verifies correct variable declaration, arithmetic evaluation, and string interpolation using the `$` prefix. The output is a `deci` (float) as specified.

#### **Test Case 6.2: If-Elif-Else Logic**
* **PPL-Script Code:**
    ```
    read "Enter score: " score(int)
    if score >= 90: print "A"
    elif score >= 80: print "B"
    elif score >= 70: print "C"
    else: print "F"
    done
    ```
* **Input & Expected Output:** If the user enters `77`, the output is `Grade: C`.
* **Correctness:** Confirms the conditional structure correctly evaluates conditions sequentially and executes only the first matching block before skipping to `done`.

#### **Test Case 6.3: `repeat as i` Loop**
* **PPL-Script Code:**
    ```
    repeat 3 as i:
        print "Loop #$i"
    done
    ```
* **Expected Output:**
    ```
    Loop #1
    Loop #2
    Loop #3
    ```
* **Correctness:** Validates the extended `repeat` loop feature, confirming that the loop executes the correct number of times and the iterator variable (`i`) is correctly exposed and updated.

---

### **Invalid Commands and Error Messages**

These examples verify the interpreter's robust error handling by ensuring invalid syntax or runtime issues terminate the program gracefully with a descriptive error message.

#### **Test Case 6.4: Syntax Error (Invalid `read` statement)**
* **PPL-Script Code:** `read "Enter value" num`
* **Expected Error Message:** `Error at line X: Invalid read syntax. Use: read "Prompt" var(type)`
* **Robustness:** The `parse` function explicitly validates the required `var(type)` structure for `read` statements, raising a `ValueError` for the syntax error rather than crashing.

#### **Test Case 6.5: Runtime Error (Division by Zero)**
* **PPL-Script Code:** `result(int) = [ 10 / 0 ]`
* **Expected Error Message:** `Error at line X: Invalid expression: 10 / 0 (division by zero)`
* **Robustness:** The `evaluate` function's `try-except` block correctly catches the underlying Python `ZeroDivisionError` and reports it as a descriptive `ValueError`, pointing to the specific line number.

#### **Test Case 6.6: Missing Expression Delimiters**
* **PPL-Script Code:** `if x > 0: print "Pos" done`
* **Expected Error Message:** `Error at line X: Invalid expression: x > 0 (name 'x' is not defined)`
* **Robustness:** Because the mandatory expression delimiters `[]` were omitted, the variable substitution logic in `evaluate` failed, leading to an underlying Python error within `eval()`. The interpreter successfully catches this error and reports it, proving the necessity of the language's defined syntax.

---

### **Conclusion on Correctness and Robustness**

These test cases prove the **correctness** of the interpreter by successfully executing commands that involve all core components: the Lexer isolates tokens, the Parser structures the commands, and the Executor performs the intended state changes and flow control. The implementation demonstrates **robustness** primarily through its dedicated error-handling strategy within the `run` and `evaluate` functions. By using `try-except` blocks and performing explicit syntax validation for specific commands, the interpreter provides clear, contextualized error messages instead of failing unexpectedly.

# Section 7. Extensions and Additional Features

Document any features added beyond the basic requirements. These extensions enhance the PPL-Script language, improving its usefulness and complexity.

### **1. Explicit Variable Typing**

* **Description:** Variables must be declared with an explicit type annotation, such as `(int)`, `(deci)` (float), or `(bool)`, unless they are assigned a string literal.
* **Implementation Detail:**
    * The `parse` function uses `re.match` to capture the type from the variable declaration string.
    * The `execute` function then performs explicit type casting (`int()`, `float()`) on the resulting value for both `assign` and `read` commands.
* **Improvement to Usefulness:** This enforces **data integrity** and reduces implicit type conversion errors, which are common issues in dynamically-typed scripting languages.

### **2. Full Conditional Structure**

* **Description:** The language supports comprehensive branching logic using `if`, `elif`, and `else` blocks, all clearly delimited and terminated by the `done` keyword.
* **Implementation Detail:** The main `run` loop contains sophisticated logic to identify the boundaries of the entire conditional block. Once an initial `if` or `elif` condition is successfully executed, the interpreter is designed to **skip over any remaining `elif` and `else` lines** until it reaches the corresponding `done` keyword.
* **Improvement to Usefulness:** This provides **essential flow control** capability for creating complex decision-making processes, mirroring standard high-level languages.

### **3. Iterative `repeat` Loop with Counter**

* **Description:** This extension introduces a convenient counted loop construct that runs a block a fixed number of times (`repeat 3:`). It includes an advanced variant that exposes an iterator variable (`repeat 3 as i:`).
* **Implementation Detail:** The `run` function uses a Python `for` loop to execute the code block lines. For the `repeat_as` extension, it temporarily writes the iteration index (`idx`) to the global `variables` dictionary using the specified loop variable name (`as_var`) before running the block.
* **Improvement to Usefulness:** This simplifies **finite iteration tasks** and provides an easily accessible loop counter without the need for manual counter variable setup, initialization, and incrementing required by a `while` loop.

### **4. Extended Arithmetic Operators**

* **Description:** The arithmetic capabilities are extended to include the Modulo (`%`) and Exponentiation (`^`) operators.
* **Implementation Detail:** The `evaluate` function handles the exponentiation operator by translating the PPL-Script symbol (`^`) to Python's valid operator (`**`) just before the expression is processed by `eval()`.
* **Improvement to Usefulness:** This enhances the language's utility for **complex mathematical and numerical computation** beyond basic addition, subtraction, multiplication, and division.

# Section 8. Insights and Conclusions

#### **Summary of Learnings**

The project successfully delivered a functional custom scripting language interpreter, providing deep, practical insight into the lifecycle of program execution. We learned that the process can be fundamentally broken down into three well-defined stages: Lexical Analysis, Syntactic Analysis, and Execution. The most critical conceptual takeaway was the decision to **delegate the complex task of expression evaluation** to the host language's built-in `eval()` function, which dramatically reduced the need to manually code operator precedence and arithmetic logic within the Executor.

#### **Strengths of the Interpreter Design**

1.  **Clear Modularity:** The strict separation of concerns into distinct functions (`tokenize`, `parse`, `execute`, `evaluate`) promotes code readability, maintainability, and ease of debugging.
2.  **Robust Flow Control:** The implementation successfully handles complex procedural logic, including nested `if-elif-else` structures and extended iterative loops (`while`, `repeat as i`).
3.  **Effective Error Reporting:** The main `run` loop uses generalized exception handling to report runtime errors with the corresponding line number, providing crucial context for the user.

#### **Limitations and Future Improvement**

1.  **Critical Limitation: Reliance on `eval()`:** The heavy reliance on Python's `eval()` for executing arbitrary expressions represents a significant security and design flaw. In a non-toy environment, `eval()` could be exploited to run unauthorized code.
    * **Future Improvement:** This must be replaced with a **custom Abstract Syntax Tree (AST) evaluation mechanism** (e.g., AST walker) to completely control the execution environment and eliminate security risks.
2.  **Global Scoping:** The current interpreter lacks support for functions and local variables, as all variables are stored in a single global dictionary (`variables`).
    * **Future Improvement:** Implement a symbol table and stack-based approach to manage **local variable scoping** for function support.
3.  **Limited Type Error Handling:** While explicit typing is required, the actual handling of type mismatch errors (e.g., attempting to add an integer to a boolean) is mostly delegated to Python's runtime errors rather than custom, user-friendly type checking messages.
    * **Future Improvement:** Implement dedicated logic within the `execute` function to validate type compatibility during assignment and arithmetic operations before calling `evaluate`.

# Section 9. References

Cite relevant references that you used in your project. All references must be cited, including:

## Scholarly Articles
- Cite in **APA format**, and put a description of how you used it for your work.

## Online References, Blogs, and Articles
- Include the website, blog, or article title, link, and how you incorporated it into your work.

## Artificial Intelligence (AI) Tools
- Put the model used (e.g., **ChatGPT**, **Gemini**).  
- Include the **complete transcript** of your conversations with the model (including your prompts and its responses).  
- Add a brief description of how you used it for your work.


#Made up language


--------------------------
Variables / data types
--------------------------

name = "Luna" //automatical sets as string when there is " "
num1(int) = 2
num2(int) = -2
money(deci) = 12.2
ans(bool) = true
ans(bool) = false


--------------------------
Arithmetic Operators
--------------------------

[add   --> + ]
[sub   --> - ]
[multiple   --> * ]
[divide   --> / ]
[module   --> % ]
[exponent   --> ^ ]


math1(int) = [ (2^2(4+4)) / (2*(8-4)) ]
math2(deci) = [ (6.7/3) * (3-4) ]

print $math1
print $math2

// output is
// 4
// -2.3333

----------------
simple math
----------------

length(deci) = 2
width(deci) = 4

area(deci) = [length * width]

print "Rectangle area = $area"

// output is
// Rectangle area = 8

------------------------
Comparison Operators
---------------------------

[equal to   --> == ]
[notequal to   --> != ]
[greater than   --> > ]
[less than   --> < ]
[greater than or equal to   --> >= ]
[less than or equal to   --> <= ]


-----------------------
Logical Operators
-----------------------

[AND   --> && ]
[OR   --> || ]



read "Enter a number: " num(int)

if [x > 0] && [x < 10]:
    print "Number is between 1 and 9"
elif [x < 0] || [x > 10]:
    print "Number is outside 0–10 range"
else:
    print "Number is 0 or 10"
done

// output is
// Enter a number: 11
// Number is outside 0–10 range

------------------------
If else statements
-------------------------

read "Enter your score: " score(int)

if score >= 90:
    print "Grade: A"
elif score >= 80:
    print "Grade: B"
elif score >= 70:
    print "Grade: C"
elif score >= 60:
    print "Grade: D"
else:
    print "Grade: F"
done

// output is
// Enter your score: 77
// Grade: C

------------------
while loop
-----------------

num(int) = 1

while [num <=5]:
    print "Number: $num"
    num = [num + 1]
done

// output is
// Number: 1
// Number: 2
// Number: 3
// Number: 4
// Number: 5


------------------
repeat loop
-----------------

repeat 3:
    print "Hi!"
done


// output is
// Hi!
// Hi!
// Hi!


repeat 3 as i:
    print "Loop #$i"
done

// output is
// Loop #1
// Loop #2
// Loop #3

#Code


In [ ]:
%%writefile rectangle.txt

read "Enter your rectangle length: " length(deci)
read "Enter your rectangle width: " width(deci)

area(deci) = (length*width)

print "The area is $area"
print $area



Writing rectangle.txt


In [ ]:
%%writefile text.txt

first = "Luna"
last = "Lynn"
print "Hello $first $last!"

Writing text.txt


In [ ]:
%%writefile math.txt

math1(int) = [ (2^2*(4+4)) / (2*(8-4)) ]

print $math1

Writing math.txt


In [ ]:
%%writefile ifelse.txt

read "Enter number: " num(int)

if [num > 0]:
    print "Positive"
elif [num == 0]:
    print "Zero"
else:
    print "Negative"
done


Writing ifelse.txt


In [ ]:
%%writefile score.txt

read "Enter your score: " score(int)

if score >= 90:
    print "Grade: A"
elif score >= 80:
    print "Grade: B"
elif score >= 70:
    print "Grade: C"
elif score >= 60:
    print "Grade: D"
else:
    print "Grade: F"
done


Writing score.txt


In [ ]:
%%writefile while.txt

num(int) = 1

while [num <=5]:
    print "Number: $num"
    num = [num + 1]
done


Writing while.txt


In [ ]:
%%writefile repeat1.txt

repeat 3:
    print "Looping!"
done


Writing repeat1.txt


In [ ]:
%%writefile repeat2.txt


repeat 3 as i:
    print $i
done


Writing repeat2.txt


In [ ]:
import sys, re

#######################
#  FILE ARGUMENT CHECK
#######################
if len(sys.argv) < 2:
    print("Usage: python interpreter.py <program_file>")
    sys.exit(1)

program_filepath = sys.argv[1]

#######################
#  LEXER
#######################

def tokenize(line):
    # remove comments
    line = re.sub(r"//.*", "", line).strip()
    if not line:
        return []
    # properly handle quoted strings and symbols
    return re.findall(r'"[^"]*"|\w+\([^)]*\)|==|!=|>=|<=|&&|\|\||[=+\-*/%^<>:]|\$?\w+', line)


#######################
#  PARSER
#######################
def parse(tokens):
    if not tokens:
        return None

    # print statement
    if tokens[0] == "print":
        return {"cmd": "print", "expr": " ".join(tokens[1:])}

    # variable declaration or math
    if "=" in tokens:
        left = tokens[0]
        value_index = tokens.index("=") + 1
        value = " ".join(tokens[value_index:])

    # support typed variable: var(type) = expr
        match = re.match(r"(\w+)\((\w+)\)", left)
        if match:
            varname, vartype = match.groups()
        else:
            varname, vartype = left, None

        return {"cmd": "assign", "name": varname, "type": vartype, "value": value}


    # read statement: read "Prompt" var(type)
    if tokens[0] == "read":
        prompt = tokens[1].strip('"')
        var_decl = tokens[2]
        match = re.match(r"(\w+)\((\w+)\)", var_decl)
        if not match:
            raise ValueError("Invalid read syntax. Use: read \"Prompt\" var(type)")
        var_name, var_type = match.groups()
        return {"cmd": "read", "prompt": prompt, "name": var_name, "type": var_type}

    # if / elif / else / done
    if tokens[0] in ["if", "elif"]:
        condition = " ".join(tokens[1:]).strip(":")
        return {"cmd": tokens[0], "condition": condition}
    if tokens[0] == "else":
        return {"cmd": "else"}
    if tokens[0] == "while":
        condition = " ".join(tokens[1:]).strip(":")
        return {"cmd": "while", "condition": condition}
    if tokens[0] == "repeat":
        # "repeat N:" or "repeat N as i:"
        if "as" in tokens:
            count = tokens[1]
            as_var = tokens[tokens.index("as") + 1].strip(":")
            return {"cmd": "repeat_as", "count": count, "as_var": as_var}
        else:
            count = tokens[1].strip(":")
            return {"cmd": "repeat", "count": count}
    if tokens[0] == "done":
        return {"cmd": "done"}

    return {"cmd": "unknown", "tokens": tokens}


#######################
#  EXECUTOR
#######################
variables = {}

def evaluate(expr):
    expr = expr.replace("^", "**")  # exponent operator
    expr = expr.replace("[", "").replace("]", "")
    for name, val in variables.items():
    # Replace $var and plain var names
        expr = expr.replace(f"${name}", str(val))
        expr = re.sub(rf"\b{name}\b", str(val), expr)

    try:
        return eval(expr)
    except Exception as e:
        raise ValueError(f"Invalid expression: {expr} ({e})")

def execute(command):
    cmd_type = command["cmd"]

    if cmd_type == "assign":
        val = evaluate(command["value"])
        if "type" in command and command["type"]:
            t = command["type"]
            if t == "int":
                val = int(val)
            elif t in ["deci", "float"]:
                val = float(val)
            elif t == "bool":
                val = bool(val)
            # strings stay as-is
        variables[command["name"]] = val


    elif cmd_type == "print":
        text = command["expr"]
        for name, val in variables.items():
            text = text.replace(f"${name}", str(val))
        print(text.strip('"'))

    elif cmd_type == "unknown":
        print(f"Unknown command: {' '.join(command['tokens'])}")

    elif cmd_type == "read":
        user_input = input(command["prompt"])
        t = command["type"]
        if t == "int":
            val = int(user_input)
        elif t in ["deci", "float"]:
            val = float(user_input)
        elif t == "bool":
            val = user_input.lower() == "true"
        else:
            val = user_input
        variables[command["name"]] = val


#######################
#  MAIN INTERPRETER
#######################
def run(filepath):
    try:
        with open(filepath, "r") as f:
            lines = [line.rstrip() for line in f.readlines()]
    except FileNotFoundError:
        print(f"Error: file '{filepath}' not found.")
        return

    i = 0
    while i < len(lines):
        line = lines[i]
        try:
            tokens = tokenize(line)
            command = parse(tokens)
            if not command:
                i += 1
                continue

            # --- Handle control structures ---
            if command["cmd"] in ["if", "elif", "else", "while", "repeat", "repeat_as"]:
                block_lines = []
                j = i + 1
                depth = 1
                while j < len(lines):
                    if lines[j].strip().startswith("done"):
                        depth -= 1
                        if depth == 0:
                            break
                    elif any(lines[j].strip().startswith(k) for k in ["if", "while", "repeat"]):
                        depth += 1
                    block_lines.append(lines[j])
                    j += 1

                if command["cmd"] == "if":
                    # Collect if/elif/else blocks until done
                    branches = []
                    branch_type = "if"
                    branch_condition = command["condition"]
                    branch_lines = []

                    j = i + 1
                    while j < len(lines):
                        line_j = lines[j].strip()
                        if line_j.startswith(("elif", "else", "done")):
                            # Save previous branch
                            branches.append((branch_type, branch_condition, branch_lines))
                            branch_lines = []

                            if line_j.startswith("elif"):
                                next_cmd = parse(tokenize(lines[j]))
                                branch_type = "elif"
                                branch_condition = next_cmd["condition"]
                            elif line_j.startswith("else"):
                                branch_type = "else"
                                branch_condition = None
                            elif line_j.startswith("done"):
                                break
                        else:
                            branch_lines.append(lines[j])
                        j += 1

                    # Now execute the correct branch
                    executed = False
                    for btype, cond, blines in branches:
                        if btype == "if" and evaluate(cond.strip("[]")):
                            run_block(blines)
                            executed = True
                            break
                        elif btype == "elif" and not executed and evaluate(cond.strip("[]")):
                            run_block(blines)
                            executed = True
                            break
                        elif btype == "else" and not executed:
                            run_block(blines)
                            executed = True
                            break

                    i = j + 1
                    continue


                elif command["cmd"] == "while":
                    while evaluate(command["condition"].strip("[]")):
                        run_block(block_lines)

                elif command["cmd"] == "repeat":
                    for _ in range(int(evaluate(command["count"]))):
                        run_block(block_lines)

                elif command["cmd"] == "repeat_as":
                    for idx in range(1, int(evaluate(command["count"])) + 1):
                        variables[command["as_var"]] = idx
                        run_block(block_lines)

                i = j + 1
                continue

            execute(command)

        except Exception as e:
            print(f"Error at line {i+1}: {e}")
        i += 1

def run_block(block_lines):
    for line in block_lines:
        tokens = tokenize(line)
        command = parse(tokens)
        if command and command["cmd"] != "done":
            execute(command)



#######################
#  ENTRY POINT
#######################
# change text file
run("ifelse.txt")





Enter number: 2
Positive
